In [21]:
import ConfigParser
import pandas as pd

# Get Configurations

All configs should be put in config.ini and placed within the same directory as this notebook.

#### Template for config.ini

[Oanda]

accountnumber = ....

apikey = ....

In [3]:
config = ConfigParser.ConfigParser()

In [4]:
config.read('config.ini')

['config.ini']

In [7]:
apikey = config.get('Oanda', 'apikey')

In [8]:
accountnumber = config.get('Oanda', 'accountnumber')

# Import Oanda Python API Wrapper

Please note that we are not using Oanda's REST API directly

Documentation: http://oanda-api-v20.readthedocs.io/en/latest/installation.html

In [11]:
import oandapyV20

In [100]:
from oandapyV20 import API
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.trades as trades

# Initiate API

In [13]:
api = API(access_token=apikey)

# Test: Get the list of all tradeable instruments

In [43]:
r = accounts.AccountInstruments(accountID=accountnumber)
rv = api.request(r)

In [48]:
tradeableIns = pd.DataFrame(rv['instruments'])
tradeableIns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 12 columns):
displayName                    124 non-null object
displayPrecision               124 non-null int64
marginRate                     124 non-null object
maximumOrderUnits              124 non-null object
maximumPositionSize            124 non-null object
maximumTrailingStopDistance    124 non-null object
minimumTradeSize               124 non-null object
minimumTrailingStopDistance    124 non-null object
name                           124 non-null object
pipLocation                    124 non-null int64
tradeUnitsPrecision            124 non-null int64
type                           124 non-null object
dtypes: int64(3), object(9)
memory usage: 11.7+ KB


In [49]:
# Well, the list is long. Let's just take a peek
tradeableIns.head(20)

,displayName,displayPrecision,marginRate,maximumOrderUnits,maximumPositionSize,maximumTrailingStopDistance,minimumTradeSize,minimumTrailingStopDistance,name,pipLocation,tradeUnitsPrecision,type
0,Gold/JPY,0,0.02,50000,0,100000,1,50,XAU_JPY,1,0,METAL
1,Taiwan Index,1,0.05,1000,0,10000.0,1,5.0,TWIX_USD,0,0,CFD
2,Gold/CAD,3,0.02,50000,0,100.000,1,0.050,XAU_CAD,-2,0,METAL
3,CAD/CHF,5,0.02,100000000,0,1.00000,1,0.00050,CAD_CHF,-4,0,CURRENCY
4,NZD/CHF,5,0.02,100000000,0,1.00000,1,0.00050,NZD_CHF,-4,0,CURRENCY
5,EUR/GBP,5,0.02,100000000,0,1.00000,1,0.00050,EUR_GBP,-4,0,CURRENCY
6,EUR/JPY,3,0.02,100000000,0,100.000,1,0.050,EUR_JPY,-2,0,CURRENCY
7,Singapore 30,2,0.02,3000,0,1000.00,1,0.50,SG30_SGD,-1,0,CFD
8,USD/CZK,5,0.05,100000000,0,1.00000,1,0.00050,USD_CZK,-4,0,CURRENCY
9,GBP/NZD,5,0.02,100000000,0,1.00000,1,0.00050,GBP_NZD,-4,0,CURRENCY


# Test: Get some pricing info

In [62]:
# Let's create a random list of instruments real quick
selectedIns = tradeableIns.sample(frac=0.2)
InsText = ','.join([name for name in selectedIns.name])
params ={'instruments': InsText}
selectedIns.count()

displayName                    25
displayPrecision               25
marginRate                     25
maximumOrderUnits              25
maximumPositionSize            25
maximumTrailingStopDistance    25
minimumTradeSize               25
minimumTrailingStopDistance    25
name                           25
pipLocation                    25
tradeUnitsPrecision            25
type                           25
dtype: int64

In [58]:
# Set  up PricingInfo
r = pricing.PricingInfo(accountID=accountnumber, params=params)

In [59]:
# Get the response
rv = api.request(r)

In [60]:
# Cache the response into Pandas DataFrame
raw_pricing = pd.DataFrame(r.response['prices'])

In [61]:
# Clean up the mess
pricing = raw_pricing.copy()
pricing.asks = pricing.asks.apply(lambda x: x[0]['price'])
pricing.bids = pricing.bids.apply(lambda x: x[0]['price'])
pricing['Available Short Units'] = pricing.unitsAvailable.apply(lambda x: x['default']['short'])
pricing['Available Long Units'] = pricing.unitsAvailable.apply(lambda x: x['default']['long'])
pricing.drop(['quoteHomeConversionFactors', 'unitsAvailable'], axis=1)

,asks,bids,closeoutAsk,closeoutBid,instrument,status,time,tradeable,type,Available Short Units,Available Long Units
0,1.79243,1.79189,1.79243,1.79189,GBP_SGD,tradeable,2017-11-22T15:30:12.337306326Z,True,PRICE,3765343,3765343
1,1.30619,1.30588,1.30619,1.30588,GBP_CHF,tradeable,2017-11-22T15:30:14.023568805Z,True,PRICE,3765343,3765343
2,4.167,4.157,4.167,4.157,WHEAT_USD,tradeable,2017-11-22T15:29:00.020300352Z,True,PRICE,100000,100000
3,8.38600,8.38390,8.38600,8.38390,USD_SEK,tradeable,2017-11-22T15:30:14.349694781Z,True,PRICE,5000000,5000000
4,23551.8,23550.2,23552.8,23549.1,US30_USD,tradeable,2017-11-22T15:30:11.329063539Z,True,PRICE,200,200
5,1094.039,1093.640,1094.039,1093.640,XAU_EUR,tradeable,2017-11-22T15:30:15.043425510Z,True,PRICE,2000,2000
6,5.35306,5.35134,5.35306,5.35134,NZD_HKD,tradeable,2017-11-22T15:30:14.917771910Z,True,PRICE,7296820,7296820
7,265.751,265.570,265.751,265.570,USD_HUF,tradeable,2017-11-22T15:30:14.647273104Z,True,PRICE,2000000,2000000
8,0.14806,0.14781,0.14806,0.14781,SUGAR_USD,tradeable,2017-11-22T15:30:10.980795219Z,True,PRICE,2000000,2000000
9,0.87233,0.87207,0.87233,0.87207,NZD_CAD,tradeable,2017-11-22T15:30:14.896329717Z,True,PRICE,7296820,7296820


# Test: Get Some Historical Prices

Singapore 30 CFD: SG30_SGD

#### Firstly, 100 prices @ 30 seconds

In [81]:
params = {'count': 20,'granularity': 'S30'}
r = instruments.InstrumentsCandles(instrument="EUR_USD",params=params)
res = api.request(r)

In [82]:
# Okie Dokie, we need to parse some info
# Let's write a little helper
def parseOHLC(df):
    temp_df = df.copy()
    temp_df['open'] = df.mid.apply(lambda x: x['o'])
    temp_df['high'] = df.mid.apply(lambda x: x['h'])
    temp_df['low'] = df.mid.apply(lambda x: x['l'])
    temp_df['close'] = df.mid.apply(lambda x: x['c'])
    temp_df.drop('mid',axis=1, inplace=True)
    temp_df = temp_df.reindex(columns = ['open','high','low','close','volume','time','complete'])
    return temp_df

parseOHLC(pd.DataFrame(res['candles']))

,open,high,low,close,volume,time,complete
0,1.17909,1.17932,1.17909,1.17925,28,2017-11-22T15:40:00.000000000Z,True
1,1.17924,1.17926,1.17910,1.17912,47,2017-11-22T15:40:30.000000000Z,True
2,1.17912,1.17916,1.17898,1.17902,43,2017-11-22T15:41:00.000000000Z,True
3,1.17902,1.17925,1.17895,1.17921,41,2017-11-22T15:41:30.000000000Z,True
4,1.17916,1.17922,1.17898,1.17912,42,2017-11-22T15:42:00.000000000Z,True
5,1.17912,1.17914,1.17910,1.17914,17,2017-11-22T15:42:30.000000000Z,True
6,1.17916,1.17930,1.17914,1.17918,15,2017-11-22T15:43:00.000000000Z,True
7,1.17914,1.17926,1.17914,1.17914,20,2017-11-22T15:43:30.000000000Z,True
8,1.17916,1.17916,1.17904,1.17904,36,2017-11-22T15:44:00.000000000Z,True
9,1.17902,1.17910,1.17896,1.17902,22,2017-11-22T15:44:30.000000000Z,True


#### 5 minutes

In [83]:
params = {'count': 20,'granularity': 'M5'}
r = instruments.InstrumentsCandles(instrument="EUR_USD",params=params)
res = api.request(r)
parseOHLC(pd.DataFrame(res['candles']))

,open,high,low,close,volume,time,complete
0,1.17582,1.17598,1.17570,1.17589,138,2017-11-22T14:15:00.000000000Z,True
1,1.17594,1.17609,1.17589,1.17595,126,2017-11-22T14:20:00.000000000Z,True
2,1.17598,1.17680,1.17593,1.17669,381,2017-11-22T14:25:00.000000000Z,True
3,1.17669,1.17746,1.17665,1.17736,551,2017-11-22T14:30:00.000000000Z,True
4,1.17738,1.17740,1.17697,1.17726,400,2017-11-22T14:35:00.000000000Z,True
5,1.17726,1.17740,1.17710,1.17734,233,2017-11-22T14:40:00.000000000Z,True
6,1.17738,1.17782,1.17736,1.17746,267,2017-11-22T14:45:00.000000000Z,True
7,1.17748,1.17780,1.17734,1.17734,184,2017-11-22T14:50:00.000000000Z,True
8,1.17731,1.17777,1.17716,1.17762,175,2017-11-22T14:55:00.000000000Z,True
9,1.17764,1.17778,1.17723,1.17723,178,2017-11-22T15:00:00.000000000Z,True


#### 1 hour

In [84]:
params = {'count': 20,'granularity': 'H1'}
r = instruments.InstrumentsCandles(instrument="EUR_USD",params=params)
res = api.request(r)
parseOHLC(pd.DataFrame(res['candles']))

,open,high,low,close,volume,time,complete
0,1.17428,1.17455,1.17350,1.17390,1096,2017-11-21T20:00:00.000000000Z,True
1,1.17387,1.17434,1.17350,1.17388,639,2017-11-21T21:00:00.000000000Z,True
2,1.17388,1.17422,1.17328,1.17342,1587,2017-11-21T22:00:00.000000000Z,True
3,1.17344,1.17476,1.17332,1.17418,878,2017-11-21T23:00:00.000000000Z,True
4,1.17412,1.17478,1.17374,1.17408,1100,2017-11-22T00:00:00.000000000Z,True
5,1.17404,1.17417,1.17356,1.17389,1011,2017-11-22T01:00:00.000000000Z,True
6,1.17392,1.17405,1.17353,1.17368,588,2017-11-22T02:00:00.000000000Z,True
7,1.17362,1.17404,1.17349,1.17386,603,2017-11-22T03:00:00.000000000Z,True
8,1.17384,1.17444,1.17381,1.17417,342,2017-11-22T04:00:00.000000000Z,True
9,1.17418,1.17454,1.17394,1.17445,426,2017-11-22T05:00:00.000000000Z,True


#### 1 Day

In [85]:
params = {'count': 20,'granularity': 'D'}
r = instruments.InstrumentsCandles(instrument="EUR_USD",params=params)
res = api.request(r)
parseOHLC(pd.DataFrame(res['candles']))

,open,high,low,close,volume,time,complete
0,1.18126,1.18370,1.16406,1.16519,70766,2017-10-25T21:00:00.000000000Z,True
1,1.16524,1.16576,1.15741,1.16080,63362,2017-10-26T21:00:00.000000000Z,True
2,1.16096,1.16578,1.15936,1.16517,39476,2017-10-29T21:00:00.000000000Z,True
3,1.16495,1.16614,1.16248,1.16458,35794,2017-10-30T21:00:00.000000000Z,True
4,1.16457,1.16575,1.16066,1.16194,45571,2017-10-31T21:00:00.000000000Z,True
5,1.16194,1.16878,1.16132,1.16584,56259,2017-11-01T21:00:00.000000000Z,True
6,1.16578,1.16914,1.15994,1.16101,46202,2017-11-02T21:00:00.000000000Z,True
7,1.16172,1.16244,1.15804,1.16096,36085,2017-11-05T22:00:00.000000000Z,True
8,1.16115,1.16157,1.15539,1.15867,42378,2017-11-06T22:00:00.000000000Z,True
9,1.15888,1.16114,1.15790,1.15944,32644,2017-11-07T22:00:00.000000000Z,True


#### 1 Week

In [86]:
params = {'count': 20,'granularity': 'W'}
r = instruments.InstrumentsCandles(instrument="EUR_USD",params=params)
res = api.request(r)
parseOHLC(pd.DataFrame(res['candles']))

,open,high,low,close,volume,time,complete
0,1.13985,1.14894,1.13706,1.14688,117042,2017-07-07T21:00:00.000000000Z,True
1,1.14741,1.16826,1.14350,1.16642,159465,2017-07-14T21:00:00.000000000Z,True
2,1.16635,1.17770,1.16128,1.17498,287891,2017-07-21T21:00:00.000000000Z,True
3,1.17482,1.19108,1.17232,1.17734,265836,2017-07-28T21:00:00.000000000Z,True
4,1.17738,1.18476,1.16888,1.18219,235670,2017-08-04T21:00:00.000000000Z,True
5,1.18220,1.18384,1.16622,1.17607,261064,2017-08-11T21:00:00.000000000Z,True
6,1.17550,1.19415,1.17314,1.19232,206794,2017-08-18T21:00:00.000000000Z,True
7,1.19458,1.20705,1.18232,1.18598,302914,2017-08-25T21:00:00.000000000Z,True
8,1.18842,1.20928,1.18684,1.20353,342462,2017-09-01T21:00:00.000000000Z,True
9,1.20179,1.20297,1.18373,1.19476,278403,2017-09-08T21:00:00.000000000Z,True


#### 1 Month

In [87]:
params = {'count': 20,'granularity': 'M'}
r = instruments.InstrumentsCandles(instrument="EUR_USD",params=params)
res = api.request(r)
parseOHLC(pd.DataFrame(res['candles']))

,open,high,low,close,volume,time,complete
0,1.13804,1.14650,1.12182,1.14497,613949,2016-03-31T21:00:00.000000000Z,True
1,1.14616,1.16164,1.10980,1.11318,485098,2016-04-30T21:00:00.000000000Z,True
2,1.11316,1.14282,1.09117,1.11062,907898,2016-05-31T21:00:00.000000000Z,True
3,1.10990,1.11975,1.09523,1.11746,564750,2016-06-30T21:00:00.000000000Z,True
4,1.11748,1.13664,1.10458,1.11582,415741,2016-07-31T21:00:00.000000000Z,True
5,1.11580,1.13272,1.11232,1.12425,470528,2016-08-31T21:00:00.000000000Z,True
6,1.12305,1.12447,1.08510,1.09820,433103,2016-09-30T21:00:00.000000000Z,True
7,1.09808,1.12998,1.05181,1.05894,821938,2016-10-31T21:00:00.000000000Z,True
8,1.05924,1.08745,1.03524,1.05198,651753,2016-11-30T22:00:00.000000000Z,True
9,1.04684,1.08124,1.03406,1.07975,800963,2016-12-31T22:00:00.000000000Z,True


# Open/Pending/Cancel orders

In [93]:
# Long EUR_USD
# This one should go through

data = {
  "order": {
    "price": "1.18",
    "instrument": "EUR_USD",
    "units": "100",
    "type": "LIMIT",
    "positionFill": "DEFAULT"
  }
}
r = orders.OrderCreate(accountnumber, data=data)
api.request(r)
pd.Series(r.response['orderFillTransaction'])

accountBalance                                          100000.0000
accountID                                       101-011-7219704-001
batchID                                                           6
commission                                                   0.0000
financing                                                    0.0000
fullPrice         {u'closeoutAsk': u'1.17910', u'timestamp': u'2...
id                                                                7
instrument                                                  EUR_USD
orderID                                                           6
pl                                                           0.0000
price                                                       1.17895
reason                                                  LIMIT_ORDER
requestID                                         42367928423664088
time                                 2017-11-22T16:05:02.613422725Z
tradeOpened                    {u'units': u'100'

In [95]:
# Long EUR_USD @ low price
# This one should NOT go through

data = {
  "order": {
    "price": "1.00",
    "instrument": "EUR_USD",
    "units": "100",
    "type": "LIMIT",
    "positionFill": "DEFAULT"
  }
}
r = orders.OrderCreate(accountnumber, data=data)
api.request(r)
pd.Series(r.response)

lastTransactionID                                                         9
orderCreateTransaction    {u'batchID': u'9', u'triggerCondition': u'DEFA...
relatedTransactionIDs                                                   [9]
dtype: object

## Check pending orders

In [103]:
r = orders.OrdersPending(accountnumber)
pending = pd.DataFrame(api.request(r)['orders'])
pending

,createTime,id,instrument,partialFill,positionFill,price,state,timeInForce,triggerCondition,type,units
0,2017-11-22T16:05:54.335555369Z,9,EUR_USD,DEFAULT_FILL,DEFAULT,1.00000,PENDING,GTC,DEFAULT,LIMIT,100
1,2017-11-22T16:05:41.833992841Z,8,EUR_USD,DEFAULT_FILL,DEFAULT,1.00000,PENDING,GTC,DEFAULT,LIMIT,100


## Cancel all pending orders

In [106]:
for order_id in pending.id:
    r = orders.OrderCancel(accountID= accountnumber, orderID=int(order_id))
    api.request(r)
    print r.response

{u'orderCancelTransaction': {u'orderID': u'9', u'userID': 7219704, u'batchID': u'10', u'reason': u'CLIENT_REQUEST', u'requestID': u'24353532814176062', u'time': u'2017-11-22T16:16:33.226877891Z', u'type': u'ORDER_CANCEL', u'id': u'10', u'accountID': u'101-011-7219704-001'}, u'lastTransactionID': u'10', u'relatedTransactionIDs': [u'10']}
{u'orderCancelTransaction': {u'orderID': u'8', u'userID': 7219704, u'batchID': u'11', u'reason': u'CLIENT_REQUEST', u'requestID': u'24353532814177039', u'time': u'2017-11-22T16:16:33.514586150Z', u'type': u'ORDER_CANCEL', u'id': u'11', u'accountID': u'101-011-7219704-001'}, u'lastTransactionID': u'11', u'relatedTransactionIDs': [u'11']}


In [107]:
r = orders.OrdersPending(accountnumber)
pending = pd.DataFrame(api.request(r)['orders'])
pending

""


# List all open trades

In [109]:
r = trades.OpenTrades(accountID=accountnumber)
api.request(r)
openTrades = pd.DataFrame(r.response['trades'])
openTrades

,currentUnits,financing,id,initialUnits,instrument,openTime,price,realizedPL,state,unrealizedPL
0,100,0.0000,7,100,EUR_USD,2017-11-22T16:05:02.613422725Z,1.17895,0.0000,OPEN,0.0690
1,100,0.0000,5,100,EUR_USD,2017-11-22T16:04:33.734302135Z,1.17895,0.0000,OPEN,0.0690


# Close all positions

In [110]:
for trade_id in openTrades.id:
    r = trades.TradeClose(accountID=accountnumber, tradeID=int(trade_id))
    api.request(r)
    print r.response

{u'orderFillTransaction': {u'orderID': u'12', u'financing': u'-0.0001', u'accountBalance': u'100000.0689', u'commission': u'0.0000', u'price': u'1.17964', u'fullPrice': {u'closeoutAsk': u'1.17992', u'timestamp': u'2017-11-22T16:20:39.486183474Z', u'closeoutBid': u'1.17949', u'bids': [{u'price': u'1.17964', u'liquidity': u'10000000'}], u'asks': [{u'price': u'1.17977', u'liquidity': u'10000000'}]}, u'userID': 7219704, u'batchID': u'12', u'instrument': u'EUR_USD', u'reason': u'MARKET_ORDER_TRADE_CLOSE', u'tradesClosed': [{u'units': u'-100', u'financing': u'-0.0001', u'realizedPL': u'0.0690', u'tradeID': u'7'}], u'time': u'2017-11-22T16:20:52.699932635Z', u'units': u'-100', u'requestID': u'24353533901340151', u'type': u'ORDER_FILL', u'id': u'13', u'pl': u'0.0690', u'accountID': u'101-011-7219704-001'}, u'orderCreateTransaction': {u'batchID': u'12', u'positionFill': u'REDUCE_ONLY', u'userID': 7219704, u'timeInForce': u'FOK', u'instrument': u'EUR_USD', u'reason': u'TRADE_CLOSE', u'tradeClose

# Check Portfolio

In [112]:
r = accounts.AccountSummary(accountnumber)
api.request(r)
pd.Series(r.response['account'])

NAV                                               100000.1378
alias                                                 Primary
balance                                           100000.1378
commission                                             0.0000
createdByUserID                                       7219704
createdTime                    2017-11-22T14:21:54.022373774Z
currency                                                  USD
financing                                             -0.0002
hedgingEnabled                                          False
id                                        101-011-7219704-001
lastTransactionID                                          15
marginAvailable                                   100000.1378
marginCallMarginUsed                                   0.0000
marginCallPercent                                     0.00000
marginCloseoutMarginUsed                               0.0000
marginCloseoutNAV                                 100000.1378
marginCl